In [2]:
import pygeohash as gh
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler

%load_ext autoreload
%autoreload 2

sys.path.append(os.path.dirname(sys.path[0]))

from uk_road_safety.python.data_cleaning import prepare_data_for_groupby, add_geohash
from uk_road_safety.python.grouped_data import data_for_analysis
from uk_road_safety.python import mlmodel
import tensorflow as tf

2022-12-11 15:14:24.927827: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-11 15:14:25.426572: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 15:14:25.426617: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-11 15:14:27.002462: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
traffic_density=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft_traffic_counts_raw_counts.csv', sep=',', header=0)

/tmp/ipykernel_30801/2633287194.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_density=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/dft_traffic_counts_raw_counts.csv', sep=',', header=0)


In [4]:
traffic_density=traffic_density.drop(['Count_point_id','Direction_of_travel','Region_id','Region_name','Region_ons_code','Local_authority_id',
                                      'Local_authority_name','Buses_and_coaches','LGVs','HGVs_2_rigid_axle','HGVs_3_rigid_axle',
                                      'HGVs_4_or_more_rigid_axle','HGVs_3_or_4_articulated_axle','HGVs_5_articulated_axle',
                                      'HGVs_6_articulated_axle','All_HGVs','Local_authority_code','Road_name','Road_category',
                                      'Road_type','Start_junction_road_name','End_junction_road_name','Easting','Northing',
                                      'Link_length_km','Link_length_miles','Pedal_cycles','Two_wheeled_motor_vehicles',
                                      'Cars_and_taxis',],axis=1)
traffic_density.head()

,Year,Count_date,hour,Latitude,Longitude,All_motor_vehicles
0,2011,2011-06-07 00:00:00,11,52.211117,-2.15655,429.0
1,2011,2011-06-07 00:00:00,12,52.211117,-2.15655,361.0
2,2011,2011-06-07 00:00:00,13,52.211117,-2.15655,477.0
3,2011,2011-06-07 00:00:00,14,52.211117,-2.15655,509.0
4,2011,2011-06-07 00:00:00,15,52.211117,-2.15655,528.0


(58038,)

In [8]:
traffic_density['geohash']=traffic_density.apply(lambda x: gh.encode(x.Latitude, x.Longitude, precision=6), axis=1)

In [9]:
traffic_density.head()

,Year,Count_date,hour,Latitude,Longitude,All_motor_vehicles,geohash
0,2011,2011-06-07 00:00:00,11,52.211117,-2.15655,429.0,gcq3nb
1,2011,2011-06-07 00:00:00,12,52.211117,-2.15655,361.0,gcq3nb
2,2011,2011-06-07 00:00:00,13,52.211117,-2.15655,477.0,gcq3nb
3,2011,2011-06-07 00:00:00,14,52.211117,-2.15655,509.0,gcq3nb
4,2011,2011-06-07 00:00:00,15,52.211117,-2.15655,528.0,gcq3nb


In [11]:
traffic_density.to_csv('~/code/MagicaBleps/UK-road-safety/raw_data/traffic_density_with_geohash.csv')

In [12]:
traffic_density['Count_date']=pd.to_datetime(traffic_density['Count_date'])

In [13]:
traffic_density.head()

,Year,Count_date,hour,Latitude,Longitude,All_motor_vehicles,geohash
0,2011,2011-06-07,11,52.211117,-2.15655,429.0,gcq3nb
1,2011,2011-06-07,12,52.211117,-2.15655,361.0,gcq3nb
2,2011,2011-06-07,13,52.211117,-2.15655,477.0,gcq3nb
3,2011,2011-06-07,14,52.211117,-2.15655,509.0,gcq3nb
4,2011,2011-06-07,15,52.211117,-2.15655,528.0,gcq3nb


In [14]:
geohashes=['gcpvj0', 'gcpvhc', 'gcpvj4', 'gcpvj1', 'gcpuv2']

In [16]:
for hash in geohashes:
    print(hash in traffic_density['geohash'].unique())

True
True
True
True
True


In [17]:
df_accidents=pd.read_csv('~/code/MagicaBleps/UK-road-safety/raw_data/preprocessed-1999-2021.csv')

In [18]:
df_accidents['date']=pd.to_datetime(df_accidents['date'])

# First Geohash gcpvj0

In [19]:
df_accidents_gcpvj0=df_accidents[df_accidents['geohash']=='gcpvj0']
time_series=data_for_analysis(df_accidents_gcpvj0,'W')

In [25]:
traffic_density_gcpvj0=traffic_density[traffic_density['geohash']=='gcpvj0'][['Count_date','All_motor_vehicles']]
#time_series_density =traffic_density_gcpvj0[['Count_date','All_motor_vehicles']].groupby('Count_date').sum()

In [30]:
foo=traffic_density_gcpvj0.set_index('Count_date').resample('W').sum()

In [31]:
foo[foo['All_motor_vehicles']>0]

,All_motor_vehicles
Count_date,
2000-05-21,25813.0
2000-06-25,31493.0
2000-07-16,22510.0
2000-10-08,48416.0
2000-10-15,26314.0
2001-06-10,13332.0
2001-09-30,23427.0
2002-04-28,9446.0
2003-06-08,17999.0
